In [14]:
from bs4 import BeautifulSoup
import requests

def search(s) : 
    movie = s
    url = f'https://movie.naver.com/movie/search/result.naver?query={movie}&section=all&ie=utf8'
    res = requests.get(url)
    index = 1
    user_dic = {}
    if res.status_code == 200:
        soup=BeautifulSoup(res.text,'lxml')
        if soup.find("ul", class_="search_list_1") is None:
            print('결과가 없습니다.')
            return ''
        for href in soup.find("ul", class_="search_list_1").find_all("li"): 
            print(f"=============={index}번 영화===============")
            print(href.dl.text[:-2])
            user_dic[index] = int(href.dl.dt.a['href'][30:])
            index = index+1
    movie_num = int(input("몇 번 영화를 선택하시겠습니까? (숫자만 입력)  : "))
    code = user_dic[movie_num]
    base_url = f'https://movie.naver.com/movie/bi/mi/pointWriteFormList.nhn?code={code}&type=after&onlyActualPointYn=N&onlySpoilerPointYn=N&order=sympathyScore&page='
    return base_url+'{}'
    #페이지 포매팅하기 위함

In [17]:
movie_list_19 = ['극한직업', '기생충', '엑시트', '봉오동전투', '나쁜녀석들:더무비', '82년생 김지영', '돈', '악인전', '가장보통의연애', '말모이',
           '증인', '블랙머니', '사바하', '타짜: 원 아이드 잭', '신의한수:귀수편', '내안의 그놈', '뺑반', '변신', '걸캅스', '사자',
           '나의 특별한 형제', '퍼펙트맨', '생일', '힘을 내요, 미스터 리', '항거:유관순이야기', '장사리:잊혀진 영웅들', '롱 리브 더 킹: 목포 영웅', '나랏말싸미', '레드슈즈'
           , '광대들:풍문조작단', '나를 찾아줘', '양자물리학', '러브 유어셀프 인 서울', '감쪽같은 그녀', '브링 더 소울: 더 무비',
           '미성년', '배심원들', '기방도령', '기묘한 가족', '악질경찰', '어린의뢰인', '비스트', '언니', '언더독']

In [10]:
url_list_19 = []

In [ ]:
for s in movie_list_19:
    url_list_19.append(search(s))

In [ ]:
for s in movie_list_19[len(url_list_19):]:
    url_list_19.append(search(s))

In [16]:
len(url_list_19)

45

In [25]:
import re
import pandas as pd
import requests
from tqdm import tqdm
from bs4 import BeautifulSoup
import time

In [76]:
import gc

In [79]:
def crawl_review(base_url):
    gc.collect()
    try:
        def no_space(text):
            text1 = re.sub('&nbsp;|&nbsp;|\n|\t|\r', '', text)
            text2 = re.sub('\n\n','', text1)
            return text2
        res = requests.get(base_url)
        if res.status_code == 200:
            soup=BeautifulSoup(res.text,'lxml')
            total = soup.select('div.score_total > strong > em')[0].text
            pages = int(total.replace(',','')[:-1]) #17,921 > 17921로 변환 후 캐스팅
            print(f"{pages}개의 페이지에서 리뷰를 모으고 있습니다.")
            time.sleep(1)
        comments = []
        stars = []
        for page in tqdm(range(1,101)):
            url = base_url.format(page)
            res = requests.get(url)
            if res.status_code == 200:
                soup=BeautifulSoup(res.text,'lxml')
                star =  soup.select('div.score_result > ul > li > div.star_score > em')
                tds = soup.select('div.score_result > ul > li > div.score_reple > p > span')
                for st in star:
                    stars.append(int(st.text))
                for cmt in tds:
                    if cmt.text != '관람객' and cmt.text !='스포일러가 포함된 감상평입니다. 감상평 보기':
                        comments.append(no_space(cmt.text))
                if(len(comments) != len(stars)):
                    print(url)
                    break
            gc.collect()
        assert len(comments) == len(stars)
        df = pd.DataFrame({"Review":comments, "Rank":stars})
        return df
    except:
        return pd.DataFrame({})
def save_and_load(dataframe):
    basepath = 'data/'
    dataframe.to_csv(basepath+'주토피아review.csv' , index= False)
    df = pdr.read_csv(basepath+'주토피아.csv')
    return df

In [54]:
search('알라딘')

==============1번 영화===============

알라딘 (Aladdin)

9.38 (참여 27621명)

모험, 가족, 판타지, 뮤지컬, 멜로/로맨스| 미국|128분 |2019
감독 : 가이 리치|출연 : 메나 마수드, 윌 스미스, 나오미 스콧


==============2번 영화===============

알라딘 2 (Alad'2)
코미디, 드라마| 프랑스|97분 |2018
감독 : 리오넬 스테케티|출연 : 케브 아담스, 자멜 드부즈, 바네사 가이드, 에릭 쥬도르, 람지 베디아


==============3번 영화===============

알라딘: 바그다드 스캔들 (Les Nouvelles aventures d'Aladin)
모험, 코미디| 프랑스|107분 |2014
감독 : 아서 벤자쿠엔|출연 : 케브 아담스, 장 폴 루브, 바네사 가이드, 오드리 라미, 에릭 쥬도르, 미셀 블랑, 윌리엄 레브길


==============4번 영화===============

알라딘 (Aladin)
모험| 인도|132분 |2009
감독 : 수조이 고쉬|출연 : 아미타브 밧찬, 리테쉬 데쉬무크

==============5번 영화===============

알라딘 (A Kid In Aladdin's Palace)
미국|89분 |1997
감독 : 로버트 L. 레비|출연 : 토마스 이안 니콜라스, 로나 미트라



몇 번 영화를 선택하시겠습니까? (숫자만 입력)  :  1


'https://movie.naver.com/movie/bi/mi/pointWriteFormList.nhn?code=163788&type=after&onlyActualPointYn=N&onlySpoilerPointYn=N&order=sympathyScore&page={}'

In [38]:
boxoffice_22_df = pd.read_csv('Data/boxoffice_22.csv', index_col='순위')
boxoffice_19_df = pd.read_csv('Data/boxoffice_19.csv', index_col='순위')

In [39]:
movie_list_22 = list(boxoffice_22_df['영화명'])
movie_list_19 = list(boxoffice_19_df['영화명'])

In [43]:
url_list_22 = []
url_list_19 = []

In [55]:
url_list_19[3]='https://movie.naver.com/movie/bi/mi/pointWriteFormList.nhn?code=163788&type=after&onlyActualPointYn=N&onlySpoilerPointYn=N&order=sympathyScore&page={}'

In [56]:
url_list_19[3]

'https://movie.naver.com/movie/bi/mi/pointWriteFormList.nhn?code=163788&type=after&onlyActualPointYn=N&onlySpoilerPointYn=N&order=sympathyScore&page={}'

In [ ]:
for s in movie_list_19:
    url_list_19.append(search(s))

In [46]:
len(url_list_19)

49

In [80]:
for i, s in enumerate(url_list_19):
    if s=='': continue
    df = crawl_review(s)
    df.to_csv(f'Data/review_2019/{i}_review.csv', encoding='utf-8')

4719개의 페이지에서 리뷰를 모으고 있습니다.


100%|█████████████████████████████████████████| 100/100 [00:37<00:00,  2.67it/s]


6889개의 페이지에서 리뷰를 모으고 있습니다.


100%|█████████████████████████████████████████| 100/100 [00:22<00:00,  4.50it/s]


2363개의 페이지에서 리뷰를 모으고 있습니다.


...

100%|█████████████████████████████████████████| 100/100 [00:18<00:00,  5.27it/s]


881개의 페이지에서 리뷰를 모으고 있습니다.


100%|█████████████████████████████████████████| 100/100 [00:21<00:00,  4.56it/s]


182개의 페이지에서 리뷰를 모으고 있습니다.


100%|█████████████████████████████████████████| 100/100 [00:18<00:00,  5.44it/s]


In [59]:
for s in movie_list_22:
    url_list_22.append(search(s))

==============1번 영화===============

범죄도시2 (the roundup)

9.30 (참여 27674명)

범죄, 액션| 대한민국|106분 |2022
감독 : 이상용|출연 : 마동석, 손석구, 최귀화




몇 번 영화를 선택하시겠습니까? (숫자만 입력)  :  1


==============1번 영화===============

탑건: 매버릭 (Top Gun: Maverick)

9.76 (참여 42786명)

액션| 미국|130분 |2021
감독 : 조셉 코신스키|출연 : 톰 크루즈, 제니퍼 코넬리, 마일즈 텔러





몇 번 영화를 선택하시겠습니까? (숫자만 입력)  :  1


==============1번 영화===============

아바타: 물의 길 (Avatar: The Way of Water)

8.65 (참여 25041명)

액션, 모험, SF, 스릴러| 미국|192분 |2022
감독 : 제임스 카메론|출연 : 시고니 위버, 스티븐 랭, 조 샐다나, 샘 워싱턴, 우나 채플린, 케이트 윈슬렛, 지오바니 리비시




...

몇 번 영화를 선택하시겠습니까? (숫자만 입력)  :  1


==============1번 영화===============

배드 가이즈 (The Bad Guys)

8.66 (참여 698명)

애니메이션, 범죄, 액션| 미국|100분 |2022
감독 : 피에르 페리펠|출연 : 샘 록웰, 아콰피나, 안소니 라모스, 마크 마론, 크레이그 로빈슨, 재지 비츠, 알렉스 볼스타인, 리차드 아요아데, 릴리 싱


==============2번 영화===============

배드 가이스 (Bad Guys)
코미디| 미국 |1986
감독 : 조엘 실버그|출연 : 아담 볼드윈, 마이크 졸리

==============3번 영화===============

어썸 아시안 배드 가이즈 (Awesome Asian Bad Guys)
코미디, 액션| 미국|52분 |2014
감독 : 패트릭 에피노, 스티븐 디피앙코|출연 : 스티븐 디피앙코, 패트릭 에피노, 탐린 토미타, 알 레옹, 조지 청

==============4번 영화===============

더 배드 가이즈 (The Bad Guys)
코미디, 드라마| 미국|90분 |2014
감독 : 카를로스 린컨즈|출연 : 글렌 포웰, 메이크너 멜빈



몇 번 영화를 선택하시겠습니까? (숫자만 입력)  :  1


==============1번 영화===============

뜨거운 피 (Hot Blooded)

7.29 (참여 2712명)

범죄, 드라마| 대한민국|120분 |2020
감독 : 천명관|출연 : 정우, 김갑수




몇 번 영화를 선택하시겠습니까? (숫자만 입력)  :  1


In [65]:
url_list_22=(url_list_22[28:])

In [68]:
url_list_22

['https://movie.naver.com/movie/bi/mi/pointWriteFormList.nhn?code=192608&type=after&onlyActualPointYn=N&onlySpoilerPointYn=N&order=sympathyScore&page={}',
 'https://movie.naver.com/movie/bi/mi/pointWriteFormList.nhn?code=81888&type=after&onlyActualPointYn=N&onlySpoilerPointYn=N&order=sympathyScore&page={}',
 'https://movie.naver.com/movie/bi/mi/pointWriteFormList.nhn?code=74977&type=after&onlyActualPointYn=N&onlySpoilerPointYn=N&order=sympathyScore&page={}',
 'https://movie.naver.com/movie/bi/mi/pointWriteFormList.nhn?code=194196&type=after&onlyActualPointYn=N&onlySpoilerPointYn=N&order=sympathyScore&page={}',
 'https://movie.naver.com/movie/bi/mi/pointWriteFormList.nhn?code=201641&type=after&onlyActualPointYn=N&onlySpoilerPointYn=N&order=sympathyScore&page={}',
 'https://movie.naver.com/movie/bi/mi/pointWriteFormList.nhn?code=182016&type=after&onlyActualPointYn=N&onlySpoilerPointYn=N&order=sympathyScore&page={}',
 'https://movie.naver.com/movie/bi/mi/pointWriteFormList.nhn?code=195758

In [81]:
for i, s in enumerate(url_list_22):
    if s=='': continue
    df = crawl_review(s)
    df.to_csv(f'Data/review_2022/{i}_review.csv', encoding='utf-8')

2761개의 페이지에서 리뷰를 모으고 있습니다.


100%|█████████████████████████████████████████| 100/100 [00:22<00:00,  4.48it/s]


4274개의 페이지에서 리뷰를 모으고 있습니다.


100%|█████████████████████████████████████████| 100/100 [00:21<00:00,  4.73it/s]


2504개의 페이지에서 리뷰를 모으고 있습니다.


...

100%|█████████████████████████████████████████| 100/100 [00:22<00:00,  4.52it/s]


69개의 페이지에서 리뷰를 모으고 있습니다.


100%|█████████████████████████████████████████| 100/100 [00:19<00:00,  5.25it/s]


270개의 페이지에서 리뷰를 모으고 있습니다.


100%|█████████████████████████████████████████| 100/100 [00:22<00:00,  4.47it/s]
